In [48]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [23]:
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv') 

In [24]:
movies.sample(3)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
4798,220000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",NaN,9367,"[{""id"": 5616, ""name"": ""united states\u2013mexi...",es,El Mariachi,El Mariachi just wants to play his guitar and ...,14.269792,"[{""name"": ""Columbia Pictures"", ""id"": 5}]","[{""iso_3166_1"": ""MX"", ""name"": ""Mexico""}, {""iso...",1992-09-04,2040920,81.0,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,"He didn't come looking for trouble, but troubl...",El Mariachi,6.6,238
330,79000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://www.lordoftherings.net/,121,"[{""id"": 603, ""name"": ""elves""}, {""id"": 606, ""na...",en,The Lord of the Rings: The Two Towers,Frodo and Sam are trekking to Mordor to destro...,106.914973,"[{""name"": ""WingNut Films"", ""id"": 11}, {""name"":...","[{""iso_3166_1"": ""NZ"", ""name"": ""New Zealand""}, ...",2002-12-18,926287400,179.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,A New Power Is Rising.,The Lord of the Rings: The Two Towers,8.0,7487
3915,0,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",NaN,53862,"[{""id"": 6270, ""name"": ""high school""}, {""id"": 1...",en,Trippin',Greg is near the end of his senior year in hig...,0.185373,"[{""name"": ""Beacon Communications"", ""id"": 919}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",1999-05-12,9016377,94.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,He's a legend in his own mind.,Trippin',4.5,4


In [25]:
movies.shape

(4803, 20)

In [26]:
credits.sample(3)

,movie_id,title,cast,crew
2551,24150,Halloween II,"[{""cast_id"": 5, ""character"": ""Laurie Strode"", ...","[{""credit_id"": ""533d2b74c3a36878e6002c1b"", ""de..."
4162,26388,Buried,"[{""cast_id"": 1, ""character"": ""Paul Conroy"", ""c...","[{""credit_id"": ""5991d5829251410448000083"", ""de..."
4494,323967,Walter,"[{""cast_id"": 37, ""character"": ""Walter"", ""credi...","[{""credit_id"": ""54d4d10592514109ba013ab1"", ""de..."


In [27]:
movies = movies.merge(credits,on='title')

In [28]:
movies.sample(3)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
2842,12000000,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 10749, ""n...",NaN,1360,"[{""id"": 437, ""name"": ""painter""}, {""id"": 2038, ...",en,Frida,"""Frida"" chronicles the life Frida Kahlo shared...",18.422327,"[{""name"": ""Miramax Films"", ""id"": 14}, {""name"":...",...,123.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Prepare to be seduced,Frida,7.3,383,1360,"[{""cast_id"": 21, ""character"": ""Frida Kahlo"", ""...","[{""credit_id"": ""52fe42eec3a36847f802dc1d"", ""de..."
796,60000000,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 37, ""nam...",http://www.netflix.com/title/80039517,347969,"[{""id"": 155573, ""name"": ""wild west""}]",en,The Ridiculous 6,"When his long-lost outlaw father returns, Tomm...",19.694695,"[{""name"": ""Happy Madison Productions"", ""id"": 2...",...,119.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,NaN,The Ridiculous 6,4.9,388,347969,"[{""cast_id"": 3, ""character"": ""Tommy aka White ...","[{""credit_id"": ""5679392c925141118700256c"", ""de..."
1907,25000000,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 10751, ""n...",NaN,15487,"[{""id"": 6075, ""name"": ""sport""}]",en,The Greatest Game Ever Played,A biopic of 20-year-old Francis Ouimet who def...,7.538513,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]",...,120.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,Based on a True Story,The Greatest Game Ever Played,6.9,110,15487,"[{""cast_id"": 1, ""character"": ""Francis Ouimet"",...","[{""credit_id"": ""52fe465f9251416c75076401"", ""de..."


In [29]:
movies = movies[['movie_id','title','overview','genres','keywords','cast','crew']]

In [30]:
import ast
def convert(text):
    L = []
    for i in ast.literal_eval(text):
        L.append(i['name']) 
    return L

In [31]:
movies.dropna(inplace=True)

In [32]:
movies['genres'] = movies['genres'].apply(convert)
movies.sample(3)

,movie_id,title,overview,genres,keywords,cast,crew
2563,14047,The Great Debaters,The true story of a brilliant but politically ...,[Drama],"[{""id"": 5565, ""name"": ""biography""}]","[{""cast_id"": 1, ""character"": ""Melvin B. Tolson...","[{""credit_id"": ""55897a9ec3a36809b1000b54"", ""de..."
2538,9587,Little Women,With their father away as a chaplain in the Ci...,"[Drama, Romance]","[{""id"": 894, ""name"": ""depression""}, {""id"": 107...","[{""cast_id"": 1, ""character"": ""Jo March"", ""cred...","[{""credit_id"": ""52fe450dc3a36847f80b9a53"", ""de..."
344,44048,Unstoppable,"A runaway train, transporting deadly, toxic ch...","[Action, Thriller]","[{""id"": 163228, ""name"": ""runaway train""}]","[{""cast_id"": 1, ""character"": ""Frank Barnes"", ""...","[{""credit_id"": ""52fe4674c3a36847f80fff55"", ""de..."


In [33]:
movies['keywords'] = movies['keywords'].apply(convert)
movies.sample(3)

,movie_id,title,overview,genres,keywords,cast,crew
4558,117942,Girls Gone Dead,A group of six ex-high school cheerleaders are...,"[Horror, Comedy]",[],"[{""cast_id"": 24, ""character"": ""Rebecca Foster""...","[{""credit_id"": ""52fe4bc4c3a36847f821312d"", ""de..."
811,1597,Meet the Parents,"Greg Focker is ready to marry his girlfriend, ...","[Comedy, Romance]","[cia, airport, cat, jew, orderly, airplane, fa...","[{""cast_id"": 1, ""character"": ""Gaylord \""Greg\""...","[{""credit_id"": ""557313ddc3a36838d3004484"", ""de..."
4529,14757,Osama,Teenage girl Osama cuts her hair and dresses l...,[Drama],[independent film],"[{""cast_id"": 2, ""character"": ""Osama"", ""credit_...","[{""credit_id"": ""565b821992514169280039fb"", ""de..."


In [34]:
ast.literal_eval('[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]')

[{'id': 28, 'name': 'Action'},
 {'id': 12, 'name': 'Adventure'},
 {'id': 14, 'name': 'Fantasy'},
 {'id': 878, 'name': 'Science Fiction'}]

In [35]:
def convert3(text):
    L = []
    counter = 0
    for i in ast.literal_eval(text):
        if counter < 3:
            L.append(i['name'])
        counter+=1
    return L 

In [36]:
movies['cast'] = movies['cast'].apply(convert)
movies.sample(3)

,movie_id,title,overview,genres,keywords,cast,crew
545,9741,Unbreakable,An ordinary man makes an extraordinary discove...,"[Science Fiction, Thriller, Drama]","[father son relationship, train accident, comi...","[Bruce Willis, Samuel L. Jackson, Robin Wright...","[{""credit_id"": ""52fe4525c3a36847f80bf021"", ""de..."
2526,873,The Color Purple,An epic tale spanning forty years in the life ...,[Drama],"[prison, africa, southern usa, rape, black peo...","[Whoopi Goldberg, Margaret Avery, Danny Glover...","[{""credit_id"": ""52fe4285c3a36847f80257ad"", ""de..."
1668,9665,Glory,Robert Gould Shaw leads the US Civil War's fir...,[War],"[war, racism, battle, union soldier, confedera...","[Matthew Broderick, Denzel Washington, Cary El...","[{""credit_id"": ""5538b05e92514138a9001d03"", ""de..."


In [37]:
movies['cast'] = movies['cast'].apply(lambda x:x[0:3])

In [38]:
def fetch_director(text):
    L = []
    for i in ast.literal_eval(text):
        if i['job'] == 'Director':
            L.append(i['name'])
    return L 

In [39]:
movies['crew'] = movies['crew'].apply(fetch_director)

In [40]:
movies.sample(5)

,movie_id,title,overview,genres,keywords,cast,crew
1866,2140,Kiss of the Dragon,"Liu Jian, an elite Chinese police officer, com...","[Action, Crime, Thriller]","[paris, prostitute, drug]","[Jet Li, Bridget Fonda, Tchéky Karyo]",[Chris Nahon]
337,47964,A Good Day to Die Hard,"Iconoclastic, take-no-prisoners cop John McCla...","[Action, Thriller]","[bomb, cia, russia, escape, courthouse, rogue,...","[Bruce Willis, Jai Courtney, Sebastian Koch]",[John Moore]
251,38745,Gulliver's Travels,Travel writer Lemuel Gulliver takes an assignm...,[Comedy],"[journalist, forbidden love, princess, royal c...","[Jack Black, Amanda Peet, Emily Blunt]",[Rob Letterman]
1562,322,Mystic River,"A story about friendship and loyalty, guilt an...","[Thriller, Crime, Drama, Mystery]","[child abuse, sexual abuse, loss of child, rep...","[Sean Penn, Tim Robbins, Kevin Bacon]",[Clint Eastwood]
1607,41233,Step Up 3D,A tight-knit group of New York City street dan...,"[Drama, Romance]","[musical, duringcreditsstinger, 3d]","[Rick Malambri, Adam G. Sevani, Joe Slaughter]",[Jon M. Chu]


In [41]:
def collapse(L):
    L1 = []
    for i in L:
        L1.append(i.replace(" ",""))
    return L1

In [42]:
movies['cast'] = movies['cast'].apply(collapse)
movies['crew'] = movies['crew'].apply(collapse)
movies['genres'] = movies['genres'].apply(collapse)
movies['keywords'] = movies['keywords'].apply(collapse)

In [43]:
movies.sample(3)

,movie_id,title,overview,genres,keywords,cast,crew
2963,11917,Saw V,Detective Hoffman is seemingly the last person...,"[Horror, Thriller, Crime]","[bloodsplatter, serialkiller, pigmask, nazi, j...","[TobinBell, CostasMandylor, ScottPatterson]",[DavidHackl]
3788,15059,The Loss of Sexual Innocence,The story of the sexual development of a filmm...,[Drama],[],"[JulianSands, SaffronBurrows, JustinChadwick]",[MikeFiggis]
4032,13007,Religulous,Commentator-comic Bill Maher plays devil's adv...,"[Comedy, Documentary]","[muslim, museum, jew, evidence, rabbi, intervi...","[BillMaher, JoseLuisDeJesusMiranda, AndrewNewb...",[LarryCharles]


In [44]:
movies['overview'] = movies['overview'].apply(lambda x:x.split())

In [45]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [46]:
new = movies.drop(columns=['overview','genres','keywords','cast','crew'])

In [47]:
new['tags'] = new['tags'].apply(lambda x: " ".join(x))
new.sample(3)

,movie_id,title,tags
2736,44214,Black Swan,"A ballet dancer wins the lead in ""Swan Lake"" a..."
751,16558,Duplicity,Two romantically-engaged corporate spies team ...
4174,14048,Man on Wire,"On August 7th 1974, French tightrope walker Ph..."


In [49]:
cv = CountVectorizer(max_features=5000,stop_words='english')

In [50]:
vector = cv.fit_transform(new['tags']).toarray()
vector.shape

(4806, 5000)

In [51]:
similarity = cosine_similarity(vector)

In [52]:
similarity

array([[1.        , 0.08964215, 0.06071767, ..., 0.02519763, 0.0277885 ,
        0.        ],
       [0.08964215, 1.        , 0.06350006, ..., 0.02635231, 0.        ,
        0.        ],
       [0.06071767, 0.06350006, 1.        , ..., 0.02677398, 0.        ,
        0.        ],
       ...,
       [0.02519763, 0.02635231, 0.02677398, ..., 1.        , 0.07352146,
        0.04774099],
       [0.0277885 , 0.        , 0.        , ..., 0.07352146, 1.        ,
        0.05264981],
       [0.        , 0.        , 0.        , ..., 0.04774099, 0.05264981,
        1.        ]])

In [54]:
def recommend(movie):
    index = new[new['title'] == movie].index[0]
    distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
    for i in distances[1:6]:
        print(new.iloc[i[0]].title)

In [55]:
# testing:
recommend('Superman')

Superman II
Superman Returns
Superman III
Superman IV: The Quest for Peace
Man of Steel
